# Group Details

## Group Name:

### Student 1:

### Student 2:

### Student 3:

# Loading Data and Preliminaries

In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [2]:
def load_array(filename, task):
    datapoint = np.load(filename)
    if task == 'task 1':
        initial_state = datapoint['initial_state']
        terminal_state = datapoint['terminal_state']
        return initial_state, terminal_state
    elif task == 'task 2' or task == 'task 3':
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        initial_state = whole_trajectory[:, 0]
        target = whole_trajectory[:, 1:, 1:]  # drop the first timepoint (second dim) and mass (last dim) for the prediction task
        return initial_state, target
    else:
        raise NotImplementedError("'task' argument should be 'task 1', 'task 2' or 'task 3'!")


In [3]:
"""
This cell gives an example of loading a datapoint with numpy for task 1.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
terminal_state: shape (n_bodies, [x, y])

"""

example = load_array('data/task 1/train/trajectory_0.npz', task='task 1')

initial_state, terminal_state = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {terminal_state.shape}')

body_idx = 2
print(f'The initial x-coordinate of the body with index {body_idx} in this trajectory was {initial_state[body_idx, 1]}')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 2)
The initial x-coordinate of the body with index 2 in this trajectory was -5.159721083543527


In [4]:

example = load_array('data/task 1/train/trajectory_1.npz', task='task 1')

initial_state, terminal_state = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {terminal_state.shape}')

shape of initial state (model input): (5, 5)
shape of terminal state (to be predicted by model): (5, 2)


In [5]:
"""
This cell gives an example of loading a datapoint with numpy for task 2 / 3.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
remaining_trajectory: shape (n_bodies, time, [x, y, v_x, v_y])

Note that for this task, you are asked to evaluate performance only with regard to the predictions of the positions (x and y).
If you use the velocity of the remaining trajectory for training,
this use should be purely auxiliary for the goal of predicting the positions [x,y] over time. 
While testing performance of your model on the test set, you do not have access to v_x and v_y of the remaining trajectory.

"""

example = load_array('data/task 2_3/train/trajectory_0.npz', task='task 2')

initial_state, remaining_trajectory = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {remaining_trajectory.shape}')

body_idx = 2
time_idx = 30
print(f'The y-coordinate of the body with index {body_idx} at time with index {time_idx} in remaining_trajectory was {remaining_trajectory[body_idx, time_idx, 1]}')

test_example = load_array('data/task 2_3/test/trajectory_900.npz', task='task 3')
test_initial_state, test_remaining_trajectory = test_example
print(f'the shape of the input of a test data example is {test_initial_state.shape}')
print(f'the shape of the target of a test data example is {test_remaining_trajectory.shape}')
print(f'values of the test data example at time {time_idx}:\n {test_remaining_trajectory[:, time_idx]}')
print('note: velocity values are unobserved (NaNs) in the test data!')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 49, 4)
The y-coordinate of the body with index 2 at time with index 30 in remaining_trajectory was -0.3861544940435097
the shape of the input of a test data example is (8, 5)
the shape of the target of a test data example is (8, 49, 4)
values of the test data example at time 30:
 [[-1.11611543  3.21149953         nan         nan]
 [-0.2865083   4.30801877         nan         nan]
 [ 1.07701594 -8.12529269         nan         nan]
 [-0.92053478  3.13709551         nan         nan]
 [-3.96308297 -4.27733589         nan         nan]
 [ 2.33945401 -8.67733599         nan         nan]
 [-4.83949085  3.67854952         nan         nan]
 [ 0.31080159 -9.74720071         nan         nan]]
note: velocity values are unobserved (NaNs) in the test data!


In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import os

In [7]:
""" #Read data from multiple files
initial_state_train=[]
terminal_state_train=[]
for i in range(900):
    data =load_array('data/task 1/train/trajectory_'+str(i)+'.npz', task='task 1')
    initial_state_train.append(data[0])
    terminal_state_train.append(data[1]) """



" #Read data from multiple files\ninitial_state_train=[]\nterminal_state_train=[]\nfor i in range(900):\n    data =load_array('data/task 1/train/trajectory_'+str(i)+'.npz', task='task 1')\n    initial_state_train.append(data[0])\n    terminal_state_train.append(data[1]) "

In [8]:
#For each intial state, measure the relative distance to the other bodies


In [9]:
#find the max length of the data
""" max_length=0
for i in range(900):
    if initial_state_train[i].shape[0]>max_length:
        max_length=initial_state_train[i].shape[0] """

' max_length=0\nfor i in range(900):\n    if initial_state_train[i].shape[0]>max_length:\n        max_length=initial_state_train[i].shape[0] '

# Data Handling and Preprocessing

In [10]:
def get_data(folder):
    file_list = os.listdir(folder)

    inputs = []
    targets = []

    for file_name in file_list:
        file_path = os.path.join(folder, file_name)
        input, target = load_array(file_path, 'task 1')
        inputs.append(torch.from_numpy(input))
        targets.append(torch.from_numpy(target))

    return inputs, targets



In [11]:
X_train, y_train = get_data('data/task 1/train')

In [12]:
X_train[0].shape

torch.Size([8, 5])

0-Mass
1-x
2-y
3-v_x
4-v_y



In [13]:
#Get the pairwise combination of the  and 4th column of X_train[0]

pairwise_vel= [(X_train[0][i][3],X_train[0][j][3]) for i in range(X_train[0].shape[0] ) for j in range(X_train[0].shape[0] ) if i!=j]

In [14]:
#Split pairwise_vel into 8 lists with 7 elements each
pairwise_vel_split = [pairwise_vel[i:i + 7] for i in range(0, len(pairwise_vel), 7)]

In [15]:
torch.stack(pairwise_vel_split[0][0])

tensor([-1.0969, -0.5787], dtype=torch.float64)

In [16]:
def get_pairwise_velocities(x):
    pairwise_velocities = []
    for i in range(len(x)):
        pairwise_velocities= [(x[i][j][3],x[i][k][3]) for j in range(x[i].shape[0] ) for k in range(x[i].shape[0] ) if j!=k]
    return pairwise_velocities

In [17]:
import numpy as np

def pad_array(data):
    # Pad the array with zeros if necessary to have 9 rows
    padded_input_data  = np.pad(data[0], ((0, 9 - data[0].shape[0]), (0, 0)), mode='constant')
    padded_target_data = np.pad(data[1], ((0, 9 - data[1].shape[0]), (0, 0)), mode='constant') 
    return padded_input_data, padded_target_data

def create_mask(data,padded_input_data):
    # Create a boolean mask array indicating the padded rows
    mask = np.ones_like(padded_input_data, dtype=bool)
    mask[data[0].shape[0]:] = False
    return mask

def pair_values(data):
    # Pair up the values of the same columns from every row with every other row
    num_rows, num_cols = data.shape
    padded_input_data = np.zeros((num_rows, num_rows, num_cols, 2))

    for i in range(num_rows):
        pair_idx = 0
        for j in range(num_rows):
            
            padded_input_data[i, pair_idx, :, 0] = data[i]
            padded_input_data[i, pair_idx, :, 1] = data[j]
            pair_idx += 1

    return padded_input_data

def get_euclidean_distance(x):
    euclidean_distances = np.zeros((len(x), len(x)))
    for i in range(len(x)):
        source_x, source_y= x[i][1], x[i][2]
        #euclidean_distance=[]
        for j in range(len(x)):
            target_x, target_y= x[j][1], x[j][2]
            euclidean_distances[i][j]=np.sqrt((source_x-target_x)**2+(source_y-target_y)**2)
            #euclidean_distance.append(np.sqrt((source_x-target_x)**2+(source_y-target_y)**2))
        #euclidean_distances.append(euclidean_distance)

    return euclidean_distances #list of lists

def target_difference(target_x, source_x):
    # target x and source x are of shape (9, 2)
    #Subtract the x and y coordinates of the target from the source
    return target_x - source_x


def process_file(file_path):
    # Main function to process a single file
    data = load_array(file_path, 'task 1')
    padded_input_data, padded_target_data = pad_array(data)
    mask = create_mask(data,padded_input_data)
    input_data = pair_values(padded_input_data)
    euclidean_distance = get_euclidean_distance(padded_input_data)
    target_data= target_difference(padded_target_data, padded_input_data[:,1:3])

    return input_data, target_data, padded_input_data, mask, euclidean_distance


In [20]:
class CustomDataset_2(Dataset):
    def __init__(self, folder):
        self.folder = folder
        self.file_list = os.listdir(folder)
        
    def __len__(self):
        return len(self.file_list)
    
    def _read_file(self, file_path):
        input_data, target_data,data, mask, distances= process_file(file_path)
        return input_data, target_data,data, mask, distances
    
    def __getitem__(self, index):
        file_path = os.path.join(self.folder, self.file_list[index])
        # Read and preprocess the data from the file
        input_data, target_data,data, mask, distance = self._read_file(file_path)
        # Return the preprocessed data
        return input_data, target_data,data, mask, distance
    


# Model Implementation

In [46]:
class DeepSet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DeepSet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),

        )
        self.aggregator = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim[0]*output_dim[1])
        )
        self.output_dim = output_dim

    def forward(self, x, mask, distance):
        batch_size = x.shape[0]
        encoded = self.encoder(x)
        aggregated = torch.mean(encoded, dim=1)
        aggregated = self.aggregator(aggregated)
        output = self.decoder(aggregated)
        output = output.view(batch_size, self.output_dim[0], self.output_dim[1])
        return output



In [68]:
import torch
import torch.nn as nn

class DeepSetRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim,activation= nn.LeakyReLU()):
        super(DeepSetRNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            activation,
            nn.Linear(hidden_dim, hidden_dim),
            activation,
            nn.Linear(hidden_dim, hidden_dim),
            activation,
            nn.Linear(hidden_dim, hidden_dim),
            
        )
        self.aggregator = nn.GRU(hidden_dim, hidden_dim, batch_first=True) # Replace with GRU

        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            activation,
            nn.Linear(hidden_dim, hidden_dim),
            activation,
            nn.Linear(hidden_dim, hidden_dim),
            activation,
            nn.Linear(hidden_dim, output_dim[0] * output_dim[1])
        )
        self.output_dim = output_dim

    def forward(self, x):
        batch_size = x.shape[0]
        encoded = self.encoder(x)
        _, aggregated = self.aggregator(encoded)  # Use the RNN aggregator
        aggregated = aggregated.squeeze(0)  # Remove the sequence dimension
        output = self.decoder(aggregated)
        output = output.view(batch_size, self.output_dim[0], self.output_dim[1])
        return output


In [69]:
input_dim = 5
hidden_dim = 64
output_dim = [9,2]  # Output x and y coordinates
batch_size = 32
num_epochs = 50

## NPE implementation

In [21]:
class NPEEncoder(nn.Module):
    def __init__(self, hidden_units):
        super(NPEEncoder, self).__init__()
        self.pairwise_layer = nn.Linear(10, hidden_units, bias=False)
        self.feedforward = nn.Sequential(
            nn.Linear(hidden_units, 50, bias=False),
            nn.ReLU(),
            nn.Linear(50, 50, bias=False),
            nn.ReLU(),
            nn.Linear(50, 50, bias=False),
            nn.ReLU(),
            nn.Linear(50, 50, bias=False),
            nn.ReLU()
        )
    
    def forward(self, x):
        x= x.to(torch.float32)
        x = self.pairwise_layer(x)
        x = self.feedforward(x)
        return x

class NPEDecoder(nn.Module):
    def __init__(self):
        super(NPEDecoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.Linear(55, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 2)
        )
    
    def forward(self, x):
        x = self.decoder(x)
        return x

In [22]:
class NPEModel(nn.Module):
    def __init__(self, hidden_units):
        super(NPEModel, self).__init__()
        self.encoder = NPEEncoder(hidden_units)
        self.decoder = NPEDecoder()
    
    def forward(self, x,unpaired_data, mask, distance, neighborhood_threshold=5):
        output=[]
        for batch in range(x.size(0)):
            batch_data= x[batch]
            batch_output=[]
            for body in range(batch_data.size(0)):
                if torch.all(mask[batch][body]==1):
                    focus_body= unpaired_data[batch][body]

                    #get index where the distance is greater than 0 but less than threshold
                    indices = np.where((distance[batch][body] > 0) & (distance[batch][body] < neighborhood_threshold))
                    

                    # Iterate through the row chunks
                    neighbor_encodings = []
                    for index in indices[0]:
                        # Process each row
                        chunk= x[batch][body][index]
                        chunk= torch.flatten(chunk)
                        chunk = chunk.squeeze()
                        processed_row = self.encoder(chunk)
                        neighbor_encodings.append(processed_row)
                    
                    # Take the sum of the neighbor encodings
                    if neighbor_encodings:
                        neighbor_encodings = torch.sum(torch.stack(neighbor_encodings), dim=0)
                    else:
                        neighbor_encodings = torch.zeros(50)
                    # Concatenate the focus body encoding with the focus_body
                    decoder_input = torch.cat((neighbor_encodings, focus_body), dim=0)
                    decoder_input= decoder_input.to(torch.float32)
                    # Decode the concatenated vector
                    decoded = self.decoder(decoder_input)
                    batch_output.append(decoded)
                else:
                    batch_output.append(torch.zeros(2))
            output.append(batch_output)
            

        return output

# Model Training

In [151]:
model = DeepSet(input_dim, hidden_dim, output_dim)

# Create the data loader
dataset = CustomDataset(data_dir)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

NameError: name 'DeepSet' is not defined

In [64]:
test_data_dir = 'data/task 1/test'
test_dataset = CustomDataset(test_data_dir)
test_data_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)



In [98]:
model = DeepSetRNN(input_dim, hidden_dim, output_dim)

# Create the data loader
dataset = CustomDataset(data_dir)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

NameError: name 'DeepSetRNN' is not defined

In [71]:
for epoch in range(num_epochs):
    model.train()
    for input_tensor, target_tensor in data_loader:
        #set input and target to float
        input = input_tensor.to('cpu').float()
        target = target_tensor.to('cpu').float()

        # Forward pass
        output = model(input).to('cpu')
        # Compute the loss
        loss = criterion(output, target)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    test_loss=0.0
    with torch.no_grad():
        for input_tensor, target_tensor in test_data_loader:
            input = input_tensor.to('cpu').float()
            target = target_tensor.to('cpu').float()
            output = model(input).to('cpu')
            test_loss += criterion(output, target_tensor).item()
    test_loss /= len(test_data_loader)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}, Test Loss: {test_loss}")

Epoch [1/50], Loss: 12.599453926086426, Test Loss: 14.747041029559664
Epoch [2/50], Loss: 15.24623966217041, Test Loss: 14.212319937163151
Epoch [3/50], Loss: 9.712862014770508, Test Loss: 15.662670619772591
Epoch [4/50], Loss: 17.36570167541504, Test Loss: 14.065373895045274
Epoch [5/50], Loss: 12.51584243774414, Test Loss: 14.187860914964032
Epoch [6/50], Loss: 9.923476219177246, Test Loss: 12.975637932418723
Epoch [7/50], Loss: 9.939854621887207, Test Loss: 12.88765626277024
Epoch [8/50], Loss: 18.01913070678711, Test Loss: 12.952148250987916
Epoch [9/50], Loss: 13.629838943481445, Test Loss: 13.140239588249504
Epoch [10/50], Loss: 12.470887184143066, Test Loss: 13.359621736682953
Epoch [11/50], Loss: 17.428478240966797, Test Loss: 13.773844002575249
Epoch [12/50], Loss: 14.444316864013672, Test Loss: 12.438155121061193
Epoch [13/50], Loss: 12.469012260437012, Test Loss: 12.827209521209065
Epoch [14/50], Loss: 9.219258308410645, Test Loss: 11.945247133675291
Epoch [15/50], Loss: 14.

In [41]:
for epoch in range(num_epochs):
    for batch in data_loader:
        print(type(batch))
        optimizer.zero_grad()
        output = model(batch)
        loss = criterion(output, batch)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

<class 'list'>


TypeError: linear(): argument 'input' (position 1) must be Tensor, not list

## NPE training

In [23]:
data_dir = 'data/task 1/train'

In [24]:
dataset = CustomDataset_2(data_dir)
data_loader = DataLoader(dataset, batch_size=100, shuffle=False)

In [25]:
test_data_dir = 'data/task 1/test'
test_dataset = CustomDataset_2(test_data_dir)
test_data_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

In [26]:
device = torch.device( 'cpu')

In [27]:
# Hyperparameters
hidden_units = 25
learning_rate = 0.001
learning_rate_decay = 0.99
iterations = 50
batch_size = 100

# Create the model
model = NPEModel(hidden_units)
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)

In [30]:
for iteration in range(iterations):
    for input_data, target_data,unpaired_data, mask, distance in data_loader:
    # Training steps
        input_data = input_data.to(torch.float32)
        unpaired_data = unpaired_data.to(torch.float32)
        mask = mask.to(torch.float32)
        distance = distance.to(torch.float32)
        target_data=target_data.to(torch.float32)
        optimizer.zero_grad()
        output = model(input_data, unpaired_data, mask, distance)
        output = torch.stack([torch.stack(lst) for lst in output], dim=0)
        
        
        loss = criterion(output, target_data)
        loss.backward()
        optimizer.step()

        # Learning rate decay
        if (iteration + 1) % 10 == 0:
            learning_rate *= learning_rate_decay
            for param_group in optimizer.param_groups:
                param_group['lr'] = learning_rate
        

        model.eval()
        test_loss=0.0
        with torch.no_grad():
            for input_data, target_data,unpaired_data, mask, distance in test_data_loader:
                input_data = input_data.to(torch.float32)
                target_data = target_data.to(torch.float32)
                unpaired_data = unpaired_data.to(torch.float32)
                mask = mask.to(torch.float32)
                distance = distance.to(torch.float32)
                output = model(input_data, unpaired_data, mask, distance)
                output = torch.stack([torch.stack(lst) for lst in output], dim=0)
                test_loss += criterion(output, target_data).item()
        test_loss /= len(test_data_loader)

                
        # Print loss for monitoring
        #if (iteration + 1) % 10000 == 0:
    print(f"Epoch [{iteration+1}/{iterations}], Loss: {loss.item()}, Test Loss: {test_loss}")

Epoch [1/50], Loss: 5.609353065490723, Test Loss: 4.0937652587890625
Epoch [2/50], Loss: 5.379310131072998, Test Loss: 3.947829484939575
Epoch [3/50], Loss: 5.259251117706299, Test Loss: 3.9337656497955322
Epoch [4/50], Loss: 5.015397071838379, Test Loss: 3.7005913257598877
Epoch [5/50], Loss: 4.872307300567627, Test Loss: 3.548375129699707
Epoch [6/50], Loss: 4.806728839874268, Test Loss: 3.460207462310791
Epoch [7/50], Loss: 4.856318473815918, Test Loss: 3.512981653213501
Epoch [8/50], Loss: 4.50195837020874, Test Loss: 3.2242136001586914
Epoch [9/50], Loss: 4.362918376922607, Test Loss: 3.1518945693969727
Epoch [10/50], Loss: 4.400338172912598, Test Loss: 3.11505126953125
Epoch [11/50], Loss: 4.286999702453613, Test Loss: 3.029372215270996
Epoch [12/50], Loss: 4.098598957061768, Test Loss: 2.9292006492614746
Epoch [13/50], Loss: 4.294901371002197, Test Loss: 2.9497263431549072
Epoch [14/50], Loss: 4.181558609008789, Test Loss: 3.2847836017608643
Epoch [15/50], Loss: 4.44419670104980

# Evaluation

In [ ]:
#todo